## FYP Sprint 3 ML training

### Ian Chia 
### 230746D

### Game Plan Idea

1) Connect to the real data (350 annotated examples)

2) Once connected train both models using the real data 

3) Compare and evaluate

--------------------

# Bert

### 1) Install & Import needed libraries

In [1]:
from dataclasses import dataclass
from typing import List, Dict, Any
import json, re, random
from pathlib import Path

import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from seqeval.metrics import precision_score, recall_score, f1_score
from jsonschema import validate, ValidationError
from transformers import TrainingArguments, Trainer

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
import json, numpy as np

C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install pymongo python-dotenv orjson

  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
   ---------------------------------------- 0.0/757.1 kB ? eta -:--:--
   ---------------------------------------- 757.1/757.1 kB 3.5 MB/s eta 0:00:00
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)

   ---------------------------------------- 0/4 [python-dotenv]
   ---------------------------------------- 0/4 [python-dotenv]
   ---------------------------------------- 0/4 [python-dotenv]
   ---------- ----------------------------- 1/4 [orjson]
   -------------------- ------------------- 2/4 [dnspython]
   -------------------- ------------------- 2/4 [dnspython]
   -------------------- ------------------- 2/4 [dnspython]
   -------------------- ------------------- 2/4 [dnspython]
   -------------------- ------------------- 2/4 [dnspython]
   -------------------- ------------------- 2/4 [dnspython]
   -------------------- ------------------- 2/4 [dnspython]
   -------------------- ------------------- 2/4 [dnspython

---------------------------

### 2) Set the connection

In [2]:
MONGO_URI = "mongodb://localhost:27017"
DB_NAME   = "FYP"
COLL_NAME = "Annotation"

----------------------

### 3) Query Mongo and preview

In [3]:
from pymongo import MongoClient

client = MongoClient(MONGO_URI)
coll = client[DB_NAME][COLL_NAME]

count = coll.count_documents({})
print("Total docs:", count)

for d in coll.find({}, {"_id": 0}).limit(3):
    print(d)


Total docs: 327
{'sentence_id': 'S001', 'sentence_name': 'None', 'original_sentence': 'A selfish, whiny teenaged girl', 'owner_id': 'ianchia', 'owner_username': 'ianchia', 'shared_with': [], 'tokens': [{'token_id': 0, 'text': 'A', 'pos': 'DET', 'label': '', 'type': ''}, {'token_id': 1, 'text': 'selfish', 'pos': 'ADJ', 'label': '', 'type': ''}, {'token_id': 2, 'text': ',', 'pos': 'PUNCT', 'label': '', 'type': ''}, {'token_id': 3, 'text': 'whiny', 'pos': 'ADJ', 'label': '', 'type': ''}, {'token_id': 4, 'text': 'teenaged', 'pos': 'ADJ', 'label': '', 'type': ''}, {'token_id': 5, 'text': 'girl', 'pos': 'NOUN', 'label': 'Entity', 'type': ''}], 'ACTIONS': [], 'ENTITIES': [{'entity_id': 'e001', 'text': 'girl', 'token_id': [5], 'ATTRIBUTES': {'POS': 'NOUN', 'ENTITY_TYPE': '', 'ADJ': {'text': 'teenaged', 'token_id': [4]}}}]}
{'sentence_id': 'S002', 'sentence_name': 'None', 'original_sentence': 'When a character delivers a speech so powerful that it emotionally moves the others to take action and

### Explanation 
- I have successfully connected to mongoDB

- I have printed a few of the document examples and I am not able to see the internal structure 

----------------------

### 4) Extract exactly what we need (text + target_json)      got error need debug


In [5]:
import orjson

def to_caseframe(doc):
    # your text is stored in "original_sentence"
    text = doc.get("original_sentence", "")

    # for now, we’ll leave the slots empty;
    # we’ll fill ACTOR/ACTION/OBJECT later once we inspect the nested structure
    frame = {
        "ACTOR": "",
        "ACTION": "",
        "OBJECT": "",
        "LOCATION": "",
        "TIME": ""
    }

    return {"text": text.strip(), "target_json": frame}

cursor = coll.find({}, {"_id": 0})
records = [to_caseframe(d) for d in cursor if (d.get("text") or d.get("sentence"))]

print("Loaded:", len(records))
print(records[0] if records else "No records!")

# save to file (T5/BERT will both use this base file later)
with open("dataset.jsonl","wb") as f:
    for r in records:
        f.write(orjson.dumps(r))
        f.write(b"\n")

print("Wrote dataset.jsonl")


Loaded: 0
No records!
Wrote dataset.jsonl


In [4]:
from pymongo import MongoClient
client = MongoClient(MONGO_URI)
coll = client[DB_NAME][COLL_NAME]

print("Total docs:", coll.count_documents({}))

Total docs: 327


In [5]:
# Show only the 'original_sentence' field for one doc
print(coll.find_one({}, {"_id":0, "original_sentence":1}))

# Show all top-level keys to confirm the exact spelling
doc = coll.find_one({}, {"_id":0})
print(sorted(list(doc.keys())))

{'original_sentence': 'A selfish, whiny teenaged girl'}
['ACTIONS', 'ENTITIES', 'original_sentence', 'owner_id', 'owner_username', 'sentence_id', 'sentence_name', 'shared_with', 'tokens']


In [6]:
has_original = coll.count_documents({"original_sentence": {"$exists": True, "$type": "string"}})
print("Docs with original_sentence string:", has_original)


Docs with original_sentence string: 327


In [7]:
import orjson

def to_caseframe(doc):
    # try multiple likely keys just in case
    text = (
        doc.get("original_sentence")
        or doc.get("sentence")
        or doc.get("text")
        or ""
    )
    if not isinstance(text, str):
        text = str(text) if text is not None else ""
    frame = {"ACTOR":"", "ACTION":"", "OBJECT":"", "LOCATION":"", "TIME":""}
    return {"text": text.strip(), "target_json": frame}

cursor = coll.find({}, {"_id": 0})
records = []
missing = 0

for d in cursor:
    r = to_caseframe(d)
    if r["text"]:
        records.append(r)
    else:
        missing += 1

print("Loaded records:", len(records), "| Missing text:", missing)
if records:
    print(records[0])

with open("dataset.jsonl","wb") as f:
    for r in records:
        f.write(orjson.dumps(r))
        f.write(b"\n")

print("Wrote dataset.jsonl")


Loaded records: 327 | Missing text: 0
{'text': 'A selfish, whiny teenaged girl', 'target_json': {'ACTOR': '', 'ACTION': '', 'OBJECT': '', 'LOCATION': '', 'TIME': ''}}
Wrote dataset.jsonl


-----------------

### 5) Peek at one doc so we know what to extract next

In [8]:
# Cell 5 — Inspect one document's structure (so we know how to pull slots)
sample = coll.find_one({}, {"_id": 0})

print("TEXT:")
print(sample.get("original_sentence", ""))

print("\nACTIONS (first 2 shown):")
print(sample.get("ACTIONS", [])[:2])

print("\nENTITIES (first 3 shown):")
print(sample.get("ENTITIES", [])[:3])

print("\nRELATIONS (first 3 shown):")
print(sample.get("RELATIONS", [])[:3])


TEXT:
A selfish, whiny teenaged girl

ACTIONS (first 2 shown):
[]

ENTITIES (first 3 shown):
[{'entity_id': 'e001', 'text': 'girl', 'token_id': [5], 'ATTRIBUTES': {'POS': 'NOUN', 'ENTITY_TYPE': '', 'ADJ': {'text': 'teenaged', 'token_id': [4]}}}]

RELATIONS (first 3 shown):
[]


### 6) Peek at tokens (so we know their structure)

In [9]:
# Cell 6 — Inspect token structure
tok = sample.get("tokens", [])

print("Token sample (first 12):")
if tok and isinstance(tok[0], dict):
    # likely a list of dicts, e.g. {"text": "...", "POS": "VERB"}
    for t in tok[:12]:
        print(t)
elif tok and isinstance(tok[0], str):
    # plain list of strings
    print(tok[:12])
else:
    print(tok)  # empty or unknown structure


Token sample (first 12):
{'token_id': 0, 'text': 'A', 'pos': 'DET', 'label': '', 'type': ''}
{'token_id': 1, 'text': 'selfish', 'pos': 'ADJ', 'label': '', 'type': ''}
{'token_id': 2, 'text': ',', 'pos': 'PUNCT', 'label': '', 'type': ''}
{'token_id': 3, 'text': 'whiny', 'pos': 'ADJ', 'label': '', 'type': ''}
{'token_id': 4, 'text': 'teenaged', 'pos': 'ADJ', 'label': '', 'type': ''}
{'token_id': 5, 'text': 'girl', 'pos': 'NOUN', 'label': 'Entity', 'type': ''}


### 7) Pick ACTION from tokens (first VERB/AUX if present)

In [12]:
# Cell 7 — Derive ACTION from tokens (first VERB/AUX if present)
def pick_action_from_tokens(token_list):
    if not token_list:
        return ""
    for t in token_list:
        p = (t.get("pos") or t.get("POS") or "").upper()
        if p in ("VERB", "AUX"):
            return (t.get("text") or "").strip()
    # no verb found → leave empty (this is okay for noun-phrase sentences)
    return ""

action_from_tokens = pick_action_from_tokens(sample.get("tokens", []))
print("ACTION:", action_from_tokens if action_from_tokens else "<none>")

ACTION: <none>


### 8) pick ACTOR from ENTITIES

In [13]:
# Cell 8 — Derive ACTOR (first entity text)
entities = sample.get("ENTITIES", [])
actor_text = ""

if entities:
    # take the first entity's text
    actor_text = (entities[0].get("text") or "").strip()

print("ACTOR:", actor_text if actor_text else "<none>")

ACTOR: girl


### 9) fill OBJECT (and leave LOCATION/TIME blank for now)

In [14]:
# Cell 9 — Derive OBJECT, keep LOCATION/TIME empty if not annotated

entities = sample.get("ENTITIES", [])

# We already picked ACTOR in Cell 8:
actor_text = (entities[0].get("text") or "").strip() if entities else ""

# OBJECT: take the next entity text if present; else empty
object_text = (entities[1].get("text") or "").strip() if len(entities) > 1 else ""

# LOCATION/TIME: if your data doesn't annotate these, leave them blank
location_text = ""
time_text = ""

print("ACTOR:", actor_text if actor_text else "<none>")
print("OBJECT:", object_text if object_text else "<none>")
print("LOCATION:", "<none>" if not location_text else location_text)
print("TIME:", "<none>" if not time_text else time_text)


ACTOR: girl
OBJECT: <none>
LOCATION: <none>
TIME: <none>


### 10) pick ACTOR / OBJECT by their ENTITY_TYPE

In [15]:
# Cell 10 — Derive ACTOR and OBJECT based on ENTITY_TYPE

entities = sample.get("ENTITIES", [])

actor_text = ""
object_text = ""

for e in entities:
    etype_raw = e.get("ATTRIBUTES", {}).get("ENTITY_TYPE") or e.get("ENTITY_TYPE") or ""
    etype = (etype_raw.get("text") if isinstance(etype_raw, dict) else etype_raw).strip().lower()
    etext = (e.get("text") or "").strip()

    # simple rule: persons or agents → ACTOR ; objects/things → OBJECT
    if not actor_text and etype in ("person", "agent", "organisation", "organization", "group"):
        actor_text = etext
    elif not object_text and etype in ("object", "thing", "item", "entity"):
        object_text = etext

# if still empty, fall back to the first / second entity positions
if not actor_text and len(entities) > 0:
    actor_text = (entities[0].get("text") or "").strip()
if not object_text and len(entities) > 1:
    object_text = (entities[1].get("text") or "").strip()

print("ACTOR:", actor_text if actor_text else "<none>")
print("OBJECT:", object_text if object_text else "<none>")


ACTOR: girl
OBJECT: <none>


### 11) Combine into one JSON frame

In [16]:
# Cell 11 — Combine into one structured case-frame JSON for this sample

frame = {
    "ACTOR": actor_text,
    "ACTION": action_from_tokens,
    "OBJECT": object_text,
    "LOCATION": "",
    "TIME": ""
}

case_frame = {"text": sample.get("original_sentence", ""), "target_json": frame}
print(case_frame)


{'text': 'A selfish, whiny teenaged girl', 'target_json': {'ACTOR': 'girl', 'ACTION': '', 'OBJECT': '', 'LOCATION': '', 'TIME': ''}}


### 12A) quick dataset audit (what do we have?)

In [17]:
# Cell 12A — Audit what’s annotated across all 327 docs

from collections import Counter

docs = list(coll.find({}, {"_id":0}))

has_actions = sum(1 for d in docs if d.get("ACTIONS"))
has_tokens_verb = 0
etype_counter = Counter()

def first_verb(tokens):
    for t in tokens or []:
        p = (t.get("pos") or t.get("POS") or "").upper()
        if p in ("VERB","AUX"):
            return True
    return False

for d in docs:
    if first_verb(d.get("tokens", [])):
        has_tokens_verb += 1
    for e in d.get("ENTITIES", []):
        et_raw = e.get("ATTRIBUTES", {}).get("ENTITY_TYPE") or e.get("ENTITY_TYPE") or ""
        et = (et_raw.get("text") if isinstance(et_raw, dict) else et_raw) or ""
        etype_counter[et.strip().lower()] += 1

print(f"Total docs: {len(docs)}")
print(f"Docs with ACTIONS list (non-empty): {has_actions}")
print(f"Docs with at least one VERB/AUX token: {has_tokens_verb}")
print("\nTop 10 ENTITY_TYPE values:")
for k, v in etype_counter.most_common(10):
    print(f"  {k or '<empty>'}: {v}")


Total docs: 327
Docs with ACTIONS list (non-empty): 262
Docs with at least one VERB/AUX token: 270

Top 10 ENTITY_TYPE values:
  <empty>: 654
  person: 93
  abstract: 26
  org: 15
  entity: 13
  object: 8
  work_of_art: 4
  place: 4
  organization: 4
  group: 2


### 13) final extractor (one function for all docs)

In [18]:
# Cell 13 — build one function to extract case-frame from a document
import orjson

def extract_caseframe(doc):
    """Return one {'text':..., 'target_json': {...}} record from a Mongo document."""
    text = (doc.get("original_sentence") or "").strip()

    # ----- ACTION -----
    actions = doc.get("ACTIONS", [])
    if actions:
        action_text = (actions[0].get("text") or "").strip()
    else:
        # fallback: pick first verb from tokens
        action_text = ""
        for t in doc.get("tokens", []):
            p = (t.get("pos") or t.get("POS") or "").upper()
            if p in ("VERB", "AUX"):
                action_text = (t.get("text") or "").strip()
                break

    # ----- ENTITY TYPES -----
    actor = object_ = location = time_ = ""
    for e in doc.get("ENTITIES", []):
        et_raw = e.get("ATTRIBUTES", {}).get("ENTITY_TYPE") or e.get("ENTITY_TYPE") or ""
        etype = (et_raw.get("text") if isinstance(et_raw, dict) else et_raw).strip().lower()
        etext = (e.get("text") or "").strip()

        if not actor and etype in ("person", "agent", "org", "organization", "group"):
            actor = etext
        elif not object_ and etype in ("object", "thing", "entity"):
            object_ = etext
        elif not location and etype in ("place", "location"):
            location = etext
        elif not time_ and etype in ("time", "date", "event"):
            time_ = etext

    # fallback positions if still empty
    ents = doc.get("ENTITIES", [])
    if not actor and len(ents) > 0:
        actor = (ents[0].get("text") or "").strip()
    if not object_ and len(ents) > 1:
        object_ = (ents[1].get("text") or "").strip()

    frame = {
        "ACTOR": actor,
        "ACTION": action_text,
        "OBJECT": object_,
        "LOCATION": location,
        "TIME": time_
    }
    return {"text": text, "target_json": frame}


### 14) apply to all docs and save to file

In [19]:
# Cell 14 — Apply extractor to all documents and save

cursor = coll.find({}, {"_id": 0})
records = [extract_caseframe(doc) for doc in cursor]

print("✅ Extracted:", len(records), "records\n")

# Preview a few samples
for r in records[:5]:
    print(orjson.dumps(r, option=orjson.OPT_INDENT_2).decode("utf-8"))
    print("-" * 60)

# Save to JSONL file for later model training
with open("dataset_full.jsonl", "wb") as f:
    for r in records:
        f.write(orjson.dumps(r))
        f.write(b"\n")

print("\n💾 Wrote all records to dataset_full.jsonl")


✅ Extracted: 327 records

{
  "text": "A selfish, whiny teenaged girl",
  "target_json": {
    "ACTOR": "girl",
    "ACTION": "",
    "OBJECT": "",
    "LOCATION": "",
    "TIME": ""
  }
}
------------------------------------------------------------
{
  "text": "When a character delivers a speech so powerful that it emotionally moves the others to take action and not lose hope",
  "target_json": {
    "ACTOR": "character",
    "ACTION": "delivers",
    "OBJECT": "speech",
    "LOCATION": "",
    "TIME": ""
  }
}
------------------------------------------------------------
{
  "text": "An ugly (in personality or appearance)/overweight character who's in romantic pursuit of another who knows they feel that way but clearly doesn't reciprocate",
  "target_json": {
    "ACTOR": "personality",
    "ACTION": "knows",
    "OBJECT": "character",
    "LOCATION": "",
    "TIME": ""
  }
}
------------------------------------------------------------
{
  "text": "sá»©c khoáº» tá»•ng Ä‘Ã n á»•n Ä‘á»‹

### 15) Load the dataset you just exported

In [20]:
# Cell 15 — Load the exported dataset
import json

records = []
with open("dataset_full.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

print("Total records loaded:", len(records))
print("Example:")
print(json.dumps(records[0], indent=2))


Total records loaded: 327
Example:
{
  "text": "A selfish, whiny teenaged girl",
  "target_json": {
    "ACTOR": "girl",
    "ACTION": "",
    "OBJECT": "",
    "LOCATION": "",
    "TIME": ""
  }
}


### 16)  Split into train / validation / test

We’ll keep ~80% for training, 10% for validation, 10% for testing.

We’ll also shuffle first so the order doesn’t matter.

In [21]:
# Cell 16 — Split dataset into train/val/test sets
import random, orjson

random.seed(42)
random.shuffle(records)

n = len(records)
train_split = int(0.8 * n)
val_split = int(0.9 * n)

train_records = records[:train_split]
val_records = records[train_split:val_split]
test_records = records[val_split:]

print(f"Train: {len(train_records)} | Validation: {len(val_records)} | Test: {len(test_records)}")


Train: 261 | Validation: 33 | Test: 33


### 17) Save the splits to separate JSONL files

In [22]:
# Cell 17 — Save the split datasets
def save_jsonl(filename, data):
    with open(filename, "wb") as f:
        for r in data:
            f.write(orjson.dumps(r))
            f.write(b"\n")

save_jsonl("train.jsonl", train_records)
save_jsonl("val.jsonl", val_records)
save_jsonl("test.jsonl", test_records)

print(" Saved train.jsonl, val.jsonl, and test.jsonl")


 Saved train.jsonl, val.jsonl, and test.jsonl


-------------------------

## Training Time

Goal: fine-tune a BERT model to label each token as part of a slot (ACTOR, ACTION, OBJECT, LOCATION, TIME).

We’ll do it in these small stages:

**Stage**	  |    **Purpose**

Cell 18	  |    install libraries

Cell 19	  |    prepare data → tokens + labels (BIO format)

Cell 20	  |     tokenize & align labels for BERT

Cell 21	  |    train BERT

Cell 22	  |    evaluate on test set (Slot-F1 & Frame-Validity)

### Cell 18 — install and import the needed libraries

In [24]:
# Cell 18A — Install missing 'evaluate' library
!pip install evaluate --quiet

In [1]:
# Cell 18 — Install and import libraries for BERT slot tagging
!pip install transformers datasets seqeval --quiet

from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments, Trainer
)
from datasets import Dataset, DatasetDict
import evaluate, numpy as np, json, orjson


C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


### Cell 19 — Convert your JSON records to token-label pairs (BIO format)

For each record:

split the sentence into tokens (simple whitespace for now),

create one label per token (B-ACTOR, I-ACTOR, O, etc.) based on the words appearing in each slot.

In [2]:
# Cell 19 — Build BIO tagging dataset for BERT
import re
from datasets import Dataset, DatasetDict

def simple_tokenize(text):
    # very simple whitespace tokenizer
    return text.split()

def make_bio_labels(text, frame):
    tokens = simple_tokenize(text)
    labels = ["O"] * len(tokens)
    for slot in ["ACTOR","ACTION","OBJECT","LOCATION","TIME"]:
        phrase = frame.get(slot, "").strip()
        if not phrase:
            continue
        phrase_tokens = phrase.split()
        # find matching subsequence of tokens in text
        for i in range(len(tokens)):
            window = tokens[i:i+len(phrase_tokens)]
            if [t.lower() for t in window] == [p.lower() for p in phrase_tokens]:
                labels[i] = f"B-{slot}"
                for j in range(1, len(phrase_tokens)):
                    labels[i+j] = f"I-{slot}"
                break
    return tokens, labels

def build_dataset_from_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            r = json.loads(line)
            text = r["text"]
            frame = r["target_json"]
            toks, labs = make_bio_labels(text, frame)
            data.append({"tokens": toks, "labels": labs})
    return Dataset.from_list(data)

train_ds = build_dataset_from_jsonl("train.jsonl")
val_ds   = build_dataset_from_jsonl("val.jsonl")
test_ds  = build_dataset_from_jsonl("test.jsonl")

datasets = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds
})

print(datasets)
print("\nExample record:")
print(datasets["train"][0])


DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 261
    })
    validation: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 33
    })
    test: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 33
    })
})

Example record:
{'tokens': ['Fat-related', 'insults'], 'labels': ['O', 'O']}


### Cell 20 — Tokenize and align labels for BERT

BERT uses its own tokenizer (wordpiece).

so we must align our word-level labels (like ["O","B-ACTOR"]) to match BERT’s subword tokens (for example, “teenaged” → “teen”, “##aged”).


In [3]:
# Cell 20 (fix) — Rebuild tokenized datasets and KEEP labels/tokens

label_list = ["O","B-ACTOR","I-ACTOR","B-ACTION","I-ACTION",
              "B-OBJECT","I-OBJECT","B-LOCATION","I-LOCATION","B-TIME","I-TIME"]

id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in id2label.items()}

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align(example):
    tokenized = tokenizer(example["tokens"], is_split_into_words=True, truncation=True, max_length=128)
    word_ids = tokenized.word_ids()
    labels = []
    for wid in word_ids:
        if wid is None:
            labels.append(-100)
        else:
            labels.append(label2id.get(example["labels"][wid], 0))
    tokenized["labels"] = labels
    # keep original tokens around for later analysis
    tokenized["orig_tokens"] = example["tokens"]
    return tokenized

tokenized_datasets = datasets.map(tokenize_and_align, batched=False)

print(tokenized_datasets)
print("Train features:", tokenized_datasets["train"].features)


C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 1026.86 examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'orig_tokens'],
        num_rows: 261
    })
    validation: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'orig_tokens'],
        num_rows: 33
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'orig_tokens'],
        num_rows: 33
    })
})
Train features: {'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'orig_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}


### Cell 21 — Train BERT (lightweight on your dataset)

In [30]:
# Cell 21A — Upgrade transformers to the latest version
!pip install -U transformers --quiet


  You can safely remove it manually.


In [31]:
# Cell 21B — fully reinstall transformers + tokenizers (Windows-safe)
import sys, os, shutil

# 1) Uninstall first
!python -m pip uninstall -y transformers tokenizers

# 2) Try to remove any half-deleted tokenizers folders (Windows sometimes locks them)
candidates = [
    r"C:\Users\ian Chia\.conda\envs\it3386\Lib\site-packages\tokenizers",
    r"C:\Users\ian Chia\.conda\envs\it3386\Lib\site-packages\~okenizers",
]
for d in candidates:
    if os.path.exists(d):
        try:
            shutil.rmtree(d, ignore_errors=True)
        except Exception as e:
            print("Could not remove", d, "->", e)

# 3) Reinstall a known-good, compatible set
!python -m pip install --no-cache-dir -q transformers==4.44.2 tokenizers==0.19.1 datasets==2.20.0 evaluate==0.4.2 accelerate==0.33.0
print("Reinstall complete.")


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
Reinstall complete.


In [4]:
# Cell 21 — Fine-tune BERT for slot tagging

from transformers import DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate, numpy as np

seqeval = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    # remove positions with -100 and map ids -> tag strings
    true_tags, pred_tags = [], []
    for p_seq, l_seq in zip(preds, labels):
        t_seq, p2_seq = [], []
        for p_id, l_id in zip(p_seq, l_seq):
            if l_id == -100:
                continue
            t_seq.append(id2label[l_id])
            p2_seq.append(id2label[p_id])
        true_tags.append(t_seq)
        pred_tags.append(p2_seq)

    results = seqeval.compute(predictions=pred_tags, references=true_tags, scheme="IOB2")
    # report overall slot-F1 (macro)
    return {
        "overall_precision": results.get("overall_precision", 0.0),
        "overall_recall": results.get("overall_recall", 0.0),
        "overall_f1": results.get("overall_f1", 0.0),
        "overall_accuracy": results.get("overall_accuracy", 0.0)
    }

data_collator = DataCollatorForTokenClassification(tokenizer)

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

args = TrainingArguments(
    output_dir="bert_slot",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=20,
    load_best_model_at_end=True,
    metric_for_best_model="overall_f1",
    greater_is_better=True,
    seed=42
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

train_result = trainer.train()
print("Training finished.")


Using the latest cached version of the module from C:\Users\ian Chia\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--seqeval\541ae017dc683f85116597d48f621abc7b21b88dc42ec937c71af5415f0af63c (last modified on Thu Nov  6 17:09:05 2025) since it couldn't be found locally at evaluate-metric--seqeval, or remotely on the Hugging Face Hub.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_mem

Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,1.006645,0.000000,0.000000,0.000000,0.729064
2,1.377000,0.877424,0.333333,0.012195,0.023529,0.729064
3,0.868200,0.805984,0.560000,0.341463,0.424242,0.780788
4,0.727600,0.774504,0.548387,0.414634,0.472222,0.785714
5,0.592200,0.793113,0.493151,0.439024,0.464516,0.780788
6,0.465900,0.802860,0.456522,0.512195,0.482759,0.773399
7,0.465900,0.820745,0.471910,0.512195,0.491228,0.775862
8,0.415600,0.854624,0.459770,0.487805,0.473373,0.773399
9,0.351900,0.844077,0.472527,0.524390,0.497110,0.778325
10,0.351900,0.854331,0.460674,0.500000,0.479532,0.775862


C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and 

Training finished.


### Cell 22 — Evaluate on the test split (Slot-F1 etc.)

In [5]:
# Cell 22 — Test-set evaluation (uses the same compute_metrics)
test_metrics = trainer.evaluate(tokenized_datasets["test"])
import json
print(json.dumps(test_metrics, indent=2))


C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{
  "eval_loss": 0.7551255822181702,
  "eval_overall_precision": 0.32558139534883723,
  "eval_overall_recall": 0.37333333333333335,
  "eval_overall_f1": 0.34782608695652173,
  "eval_overall_accuracy": 0.7869565217391304,
  "eval_runtime": 2.2882,
  "eval_samples_per_second": 14.422,
  "eval_steps_per_second": 1.311,
  "epoch": 10.0
}


C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Cell 23 — Turn BIO → frames and compute Frame-Validity %

Definition we’ll use (suited to your annotations):

A frame is “valid” if at least one of ACTOR / ACTION / OBJECT is non-empty (LOCATION/TIME are optional).

In [6]:
# Cell 23 — BIO → frame, Frame-Validity %, and a few examples

import numpy as np

def ids_to_tags(pred_ids, label_ids):
    """Remove -100 and map ids -> tag strings."""
    pred_tags, true_tags = [], []
    for p_seq, l_seq in zip(pred_ids, label_ids):
        seq_p, seq_t = [], []
        for p_id, l_id in zip(p_seq, l_seq):
            if l_id == -100:
                continue
            seq_p.append(id2label[p_id])
            seq_t.append(id2label[l_id])
        pred_tags.append(seq_p); true_tags.append(seq_t)
    return pred_tags, true_tags

def bio_to_frame(tokens, tags):
    slots = {k:"" for k in ["ACTOR","ACTION","OBJECT","LOCATION","TIME"]}
    cur_slot, buf = None, []
    for tok, tag in zip(tokens, tags):
        if tag.startswith("B-"):
            if cur_slot and buf:
                slots[cur_slot] = " ".join(buf)
            cur_slot, buf = tag[2:], [tok]
        elif tag.startswith("I-") and cur_slot == tag[2:]:
            buf.append(tok)
        else:
            if cur_slot and buf:
                slots[cur_slot] = " ".join(buf)
            cur_slot, buf = None, []
    if cur_slot and buf:
        slots[cur_slot] = " ".join(buf)
    return slots

def frame_valid(slots):
    # Valid if at least one of the core slots is non-empty
    return any(slots[k].strip() for k in ["ACTOR","ACTION","OBJECT"])

# 1) get predictions
pred = trainer.predict(tokenized_datasets["test"])
pred_ids = np.argmax(pred.predictions, axis=-1)
true_ids = pred.label_ids

# 2) ids → tag strings (strip -100)
pred_tags, true_tags = ids_to_tags(pred_ids, true_ids)

# 3) rebuild tokens to convert BIO → frames
tokens_list = tokenized_datasets["test"]["orig_tokens"]

# 4) compute Frame-Validity %
valid_count = 0
frames_pred, frames_true = [], []
for toks, p_tags, t_tags in zip(tokens_list, pred_tags, true_tags):
    p_frame = bio_to_frame(toks, p_tags)
    t_frame = bio_to_frame(toks, t_tags)
    frames_pred.append(p_frame); frames_true.append(t_frame)
    if frame_valid(p_frame):
        valid_count += 1

frame_validity = valid_count / len(tokens_list)
print(f"Frame-Validity %: {frame_validity*100:.1f}%")

# 5) show a few examples (pred vs gold)
for i in range(3):
    print("\nText:", " ".join(tokens_list[i]))
    print("Pred:", frames_pred[i])
    print("Gold:", frames_true[i])


Frame-Validity %: 81.8%

Text: Explains the standard terminology for different ways various works use tropes
Pred: {'ACTOR': 'tropes', 'ACTION': 'use', 'OBJECT': 'ways', 'LOCATION': '', 'TIME': ''}
Gold: {'ACTOR': 'ways', 'ACTION': 'Explains', 'OBJECT': 'works', 'LOCATION': '', 'TIME': ''}

Text: Pigs with pneumonia
Pred: {'ACTOR': 'Pigs', 'ACTION': '', 'OBJECT': '', 'LOCATION': '', 'TIME': ''}
Gold: {'ACTOR': 'Pigs', 'ACTION': '', 'OBJECT': 'pneumonia', 'LOCATION': '', 'TIME': ''}

Text: Flashbacks in a work are always black and white to represent a specific time period
Pred: {'ACTOR': 'work', 'ACTION': '', 'OBJECT': '', 'LOCATION': '', 'TIME': ''}
Gold: {'ACTOR': 'Flashbacks', 'ACTION': 'represent', 'OBJECT': 'work', 'LOCATION': '', 'TIME': ''}


## Result Explanation

**Slot-F1 = 0.35**

This metric tells you how accurately each slot (ACTOR, ACTION, OBJECT, etc.) was tagged at the token level.

0.35 means the model is identifying some slots correctly, but it’s also making quite a few mistakes.

This is normal and even expected for your current dataset because:

your annotations mostly label entities but not specific roles,

ACTION slots are sparse (many sentences have no verbs tagged),

the dataset is small (327 sentences total).

So the model is learning some patterns but not yet strong generalization.

**Frame-Validity = 81.8%**

This measures: how often the model outputs a structured, meaningful frame (i.e., at least one of ACTOR, ACTION, or OBJECT is filled in).

81.8% means that in ~4 out of 5 sentences, the model generated a valid frame instead of an empty or nonsense output.

This is quite good for such a small and noisy dataset.

It means your pipeline (data → model → JSON reconstruction) is working correctly, and the model is producing coherent results most of the time.

**Overall Accuracy = 79%**

### Conclusion

The BERT model was trained on 327 annotated sentences.
It achieved a Slot-F1 of 0.35, indicating partial learning of role boundaries, and a Frame-Validity of 81.8%, showing that most outputs followed a valid frame structure.
These results suggest that the current annotation coverage provides a solid proof-of-concept pipeline but requires finer role distinctions (especially for ACTION vs ENTITY types) to improve accuracy.
The system demonstrates feasibility for automated text-to-frame conversion and provides a baseline for comparison with the T5 model.

----------------------------

# T5


### Cell T1 — imports for T5

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import json, orjson

C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


###  Cell T2 — load your splits

In [2]:
# Cell T2 — load train/val/test (same files you saved earlier)
train_ds = load_dataset("json", data_files="train.jsonl", split="train")
val_ds   = load_dataset("json", data_files="val.jsonl",   split="train")
test_ds  = load_dataset("json", data_files="test.jsonl",  split="train")

print(train_ds[0])
print(len(train_ds), len(val_ds), len(test_ds))


Generating train split: 261 examples [00:00, 5192.92 examples/s]
Generating train split: 33 examples [00:00, 1245.84 examples/s]
Generating train split: 33 examples [00:00, 1609.05 examples/s]

{'text': 'Fat-related insults', 'target_json': {'ACTOR': 'Fat - related insults', 'ACTION': 'related', 'OBJECT': '', 'LOCATION': '', 'TIME': ''}}
261 33 33


### Cell T3 — build input/target strings

In [3]:
# Cell T3 — map each row to input_text and target_text
def to_io(example):
    tgt = example["target_json"]
    example["input_text"]  = f"extract slots: {example['text']}"
    example["target_text"] = json.dumps({
        "ACTOR":   tgt.get("ACTOR",""),
        "ACTION":  tgt.get("ACTION",""),
        "OBJECT":  tgt.get("OBJECT",""),
        "LOCATION":tgt.get("LOCATION",""),
        "TIME":    tgt.get("TIME","")
    }, ensure_ascii=False, separators=(",",":"))
    return example

train_io = train_ds.map(to_io)
val_io   = val_ds.map(to_io)
test_io  = test_ds.map(to_io)

print(train_io[0]["input_text"])
print(train_io[0]["target_text"])


Map: 100%|███████████████████████████████████████████████████| 33/33 [00:00<00:00, 851.83 examples/s]

extract slots: Fat-related insults
{"ACTOR":"Fat - related insults","ACTION":"related","OBJECT":"","LOCATION":"","TIME":""}


### Cell T4 — tokenizer + model + tokenization fn

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model_name = "t5-small"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def tok_fn(batch):
    # encode input
    model_inputs = tok(batch["input_text"], truncation=True, max_length=256)
    # encode target
    with tok.as_target_tokenizer():
        labels = tok(batch["target_text"], truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tok = train_io.map(tok_fn, batched=True, remove_columns=train_io.column_names)
val_tok   = val_io.map(tok_fn,   batched=True, remove_columns=val_io.column_names)
test_tok  = test_io.map(tok_fn,  batched=True, remove_columns=test_io.column_names)

data_collator = DataCollatorForSeq2Seq(tok, model=model)

print(train_tok[0].keys())

C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ian Chia\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. F

dict_keys(['input_ids', 'attention_mask', 'labels'])


### Cell T5 — training args + trainer

In [5]:
# Cell T5 — training setup
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    output_dir="t5_slots",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    predict_with_generate=True,
    generation_max_length=128,
    logging_steps=20,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    seed=42
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tok,
    data_collator=data_collator,
)
print("Trainer ready.")


C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Trainer ready.


### Cell T6 — train

In [6]:
# Cell T6 — fine-tune T5
train_result = trainer.train()
print("Training finished.")

C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,0.587821
2,1.895100,0.396432
3,0.561700,0.376771
4,0.354600,0.339412
5,0.276300,0.331424
6,0.225400,0.335627
7,0.225400,0.342082
8,0.190300,0.311399
9,0.157100,0.337047
10,0.144400,0.343844


C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ian Chia\.conda\envs\it3386\lib\site-packag

Training finished.


### Cell T7 — evaluate on test (generate → metrics)

In [11]:
# Cell T7 — Evaluate T5 on test set (robust JSON + metrics)
import torch, json, re
from pydantic import BaseModel, ValidationError

# ----- schema -----
class Frame(BaseModel):
    ACTOR: str
    ACTION: str
    OBJECT: str
    LOCATION: str
    TIME: str

# ----- helpers -----
def norm(x):
    return re.sub(r"\s+", " ", (x or "")).strip().lower()

def safe_json(s):
    s = (s or "").strip().replace("<pad>", "").replace("</s>", "").replace("<s>", "").strip()
    # wrap loose key/value pairs (no braces) like "ACTOR":"x","ACTION":"y",...
    if s and not s.lstrip().startswith("{") and "\"ACTOR\"" in s:
        s = "{" + s + "}"
    # normalize smart quotes, remove trailing commas before } or ]
    s = s.replace("“", "\"").replace("”", "\"").replace("’", "'")
    s = re.sub(r",\s*([}\]])", r"\1", s)
    try:
        return json.loads(s), True
    except Exception:
        return {}, False

def slot_f1_and_validity(pred_strs, gold_objs):
    tp = fp = fn = 0
    valid = 0
    core = ["ACTOR", "ACTION", "OBJECT"]
    all_slots = ["ACTOR","ACTION","OBJECT","LOCATION","TIME"]

    for ps, g in zip(pred_strs, gold_objs):
        pobj, ok = safe_json(ps)

        # structural validity (keys exist) + non-empty core
        if ok:
            try:
                Frame(**{k: pobj.get(k, "") for k in all_slots})
                structural_ok = True
            except ValidationError:
                structural_ok = False
        else:
            structural_ok = False

        nonempty_core = all(norm(pobj.get(k, "")) != "" for k in core)
        if structural_ok and nonempty_core:
            valid += 1

        # micro Slot-F1 over the 5 slots (exact string match after normalization)
        for k in all_slots:
            p = norm(pobj.get(k, ""))
            t = norm(g.get(k, ""))
            if p == "" and t == "":
                continue
            if p != "" and t != "":
                if p == t:
                    tp += 1
                else:
                    fp += 1
                    fn += 1
            elif p != "" and t == "":
                fp += 1
            elif p == "" and t != "":
                fn += 1

    prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    rec  = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1   = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0
    return f1, valid / max(1, len(gold_objs))

# ----- prepare inputs -----
texts = [ex["input_text"] for ex in test_io]
enc = tok(texts, return_tensors="pt", padding=True, truncation=True, max_length=256)

device = next(model.parameters()).device if hasattr(model, "parameters") else torch.device("cpu")
enc = {k: v.to(device) for k, v in enc.items()}

# ----- generate -----
with torch.no_grad():
    out_ids = model.generate(
        input_ids=enc["input_ids"],
        attention_mask=enc.get("attention_mask", None),
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
pred_texts = tok.batch_decode(out_ids, skip_special_tokens=True)

# ----- load gold -----
gold_objs = []
for x in test_io:
    tgt = x.get("target_text", "")
    try:
        gold_objs.append(json.loads(tgt))
    except Exception:
        # very lenient fallback (handles tiny formatting quirks)
        obj, ok = safe_json(tgt)
        gold_objs.append(obj if ok else {"ACTOR":"","ACTION":"","OBJECT":"","LOCATION":"","TIME":""})

# ----- metrics -----
slot_f1, frame_valid = slot_f1_and_validity(pred_texts, gold_objs)
print(f"Slot-F1: {slot_f1:.3f}")
print(f"Frame-Validity %: {frame_valid*100:.1f}%")

# ----- quick peek -----
n_show = min(5, len(test_io))
for i in range(n_show):
    print("\nText:", test_io[i]["input_text"].replace("extract slots: ",""))
    print("Pred:", pred_texts[i])
    print("Gold:", test_io[i]["target_text"])


Slot-F1: 0.374
Frame-Validity %: 57.6%

Text: Explains the standard terminology for different ways various works use tropes
Pred: "ACTOR":"standard terminology","ACTION":"Explics","OBJECT":"variable ways","LOCATION":"","TIME":""
Gold: {"ACTOR":"ways","ACTION":"Explains","OBJECT":"works","LOCATION":"","TIME":""}

Text: Pigs with pneumonia
Pred: "ACTOR":"Pigs","ACTION":"","OBJECT":" pneumonia","LOCATION":"","TIME":""
Gold: {"ACTOR":"Pigs","ACTION":"","OBJECT":"pneumonia","LOCATION":"","TIME":""}

Text: Flashbacks in a work are always black and white to represent a specific time period
Pred: "ACTOR":"work","ACTION":"represent","OBJECT":"black and white","LOCATION":"","TIME":""
Gold: {"ACTOR":"Flashbacks","ACTION":"represent","OBJECT":"work","LOCATION":"","TIME":""}

Text: The herd's health is good, so their feed intake exceeds the standard.
Pred: "ACTOR":"herd's health","ACTION":"exceeds","OBJECT":"feed intake","LOCATION":"","TIME":""
Gold: {"ACTOR":"standard","ACTION":"exceeds","OBJECT":

## Result Explanation

**Slot-F1 = 0.374**

This metric tells you how accurately each slot (ACTOR, ACTION, OBJECT, LOCATION, TIME) was predicted in the generated JSON output.

A Slot-F1 of 0.374 means the model is identifying more slots correctly than before, but still makes notable mistakes such as swapping roles or producing near-miss words (e.g., Explics instead of Explains).

This result is reasonable for your current dataset because:

The dataset remains small (327 sentences total).

Many sentences are short and lack clear ACTION or LOCATION cues.

The model must learn to both understand the sentence and generate valid JSON strings — a harder task than token tagging.

So, the T5 model is capturing some semantic structure but still generalizes inconsistently, especially on rarer slots.

**Frame-Validity = 57.6 %**

This measures how often the model outputs a fully structured and valid frame (i.e., ACTOR, ACTION, and OBJECT are all non-empty).

A score of 57.6 % means that in roughly 3 out of 5 sentences, T5 generated a usable JSON frame; the remaining cases had formatting issues or missing slots.

This lower validity rate is expected because text-generation models like T5 sometimes produce incomplete or slightly malformed outputs without decoding constraints.

It shows that while the model understands the relationships conceptually, it still struggles to enforce consistent structure.

**Overall Summary**

T5 improves Slot-F1 compared to BERT (0.374 vs 0.35), showing better per-slot accuracy.

BERT still leads in Frame-Validity (81.8 % vs 57.6 %), meaning its outputs are structurally more reliable.

**Interpretation**

The T5 model demonstrates promising flexibility — it can directly generate structured frames from text — but at the cost of output stability.
Its higher Slot-F1 suggests it learned useful mappings between text and slot meanings, while the drop in Frame-Validity highlights that generation quality still needs refinement (e.g., stricter decoding or post-processing).

-------------------------

## Comparison: BERT vs T5


**Explanation:**

Both models were trained on the same dataset of 327 annotated sentences, but they use different strategies for extracting frames.

**BERT** learns at the token level. It tags each word as ACTOR, ACTION, OBJECT, etc., and then reconstructs frames from those tags.

- Strength: Produces more structurally valid outputs (81.8 %).

- Limitation: Slightly lower Slot-F1 (0.35) due to local boundary errors and limited context.

**T5** directly generates structured JSON outputs from raw sentences.

- Strength: Achieved a higher Slot-F1 (0.374), showing better overall slot accuracy.

- Limitation: Lower Frame-Validity (57.6 %), as some generated frames were incomplete or malformed.

**Summary:**

T5 demonstrates better slot-level understanding but is less consistent structurally.
BERT, while slightly less accurate on slot values, maintains stronger control over frame format and validity.
This shows a clear trade-off between flexibility (T5) and structural reliability (BERT).

----------------------------------------------